In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops

In [2]:
input_dir = r'D:\MED_LEAF_ID\data\preprocessed_glcm'
distances = [1, 2, 3, 4, 5]  # Example distances
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Example angles in radians
allowed_extensions = {'.jpg', '.png', '.jpeg'}
features_list = []

In [3]:
# Walk through all directories and files
for root, dirs, files in os.walk(input_dir):
    for filename in files:
        if os.path.splitext(filename)[1].lower() in allowed_extensions:
            img_path = os.path.join(root, filename)

            # Extract class name from the folder structure (parent directory)
            class_name = os.path.basename(root)

            # Read image as grayscale
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"❌ Skipping unreadable image: {img_path}")
                continue

            # Normalize the image to range [0, 255]
            image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)

            # Extract GLCM features for each combination of distance and angle
            glcm_features = []

            for d in distances:
                for angle in angles:
                    # Compute GLCM matrix
                    glcm = graycomatrix(image, distances=[d], angles=[angle], symmetric=True, normed=True)

                    # Extract GLCM properties
                    contrast = graycoprops(glcm, 'contrast')[0, 0]
                    energy = graycoprops(glcm, 'energy')[0, 0]
                    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
                    correlation = graycoprops(glcm, 'correlation')[0, 0]
                    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
                    asm = graycoprops(glcm, 'ASM')[0, 0]
                    entropy = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))

                    # Append features for this (d, angle) pair
                    glcm_features.extend([contrast, energy, homogeneity, correlation, dissimilarity, asm, entropy])

            # Add the class name and features of this image to the list
            features_list.append([class_name, filename] + glcm_features)


In [6]:
# Define the columns for the DataFrame
columns = ['Class_Name', 'Image_Name']  # First two columns: class name and image name
for d in distances:
    for angle in angles:
        columns += [
            f'contrast_d{d}_a{angle}',
            f'energy_d{d}_a{angle}',
            f'homogeneity_d{d}_a{angle}',
            f'correlation_d{d}_a{angle}',
            f'dissimilarity_d{d}_a{angle}',
            f'asm_d{d}_a{angle}',
            f'entropy_d{d}_a{angle}'
        ]

# Create a DataFrame from the list of features
df = pd.DataFrame(features_list, columns=columns)


In [7]:
# Save the DataFrame to a CSV file
output_file = r'D:\MED_LEAF_ID\data\glcm_features.csv'
df.to_csv(output_file, index=False)

print(f"Features extracted and saved to {output_file}")

Features extracted and saved to D:\MED_LEAF_ID\data\glcm_features.csv
